# 将数据从SQLite数据库导出为XLIFF翻译文件

修改路径后依次运行下列代码块即可。适用于Linux环境，Windows环境下未经测试。

In [6]:
from lxml import etree
from sqlalchemy import create_engine, text
import os
from datetime import datetime

WORKDIR = "/mnt/f/OneDrive/cs/mtgdatabase/weblate_database/mtgzh"
DATABASEDIR = "/mnt/f/OneDrive/cs/mtgdatabase/"

ATOMIC_SQL_TEXT = """SELECT 
    scryfallId, 
    atomicCards.*,
    COALESCE(sets.parentCode, cards.setCode) AS parent_code,
    COALESCE(parentSets.releaseDate, sets.releaseDate) AS releaseDate,
    NULL,
    NULL
FROM 
    atomicCards 
INNER JOIN 
    cards 
ON 
    atomicCards.name = COALESCE(cards.faceName, cards.name) 
    AND atomicCards.setCode = cards.setCode 
    AND atomicCards.number = cards.number 
INNER JOIN 
    cardIdentifiers 
ON 
    cardIdentifiers.uuid = cards.uuid
INNER JOIN
    sets
ON 
    sets.code = cards.setCode
LEFT JOIN 
    sets parentSets 
ON 
    parentSets.code = sets.parentCode"""

ALL_SQL_TEXT = """SELECT 
    scryfallId, 
    COALESCE(c.faceName, c.name) AS name,
    NULL,
    c.setCode,
    c.number,
    COALESCE(c.originalText, c.text) AS text,
    NULL,
    COALESCE(z.faceName, z.name) AS zhsName,
    z.text AS zhsText,
    NULL,
    NULL,
    NULL,
    NULL,
    NULL,
    NULL,
    z.extra,
    COALESCE(sets.parentCode, c.setCode) AS parent_code,
    COALESCE(parentSets.releaseDate, sets.releaseDate) AS releaseDate,
    c.flavorText,
    z.flavorText AS zhsFlavorText
FROM 
    cards c 
LEFT JOIN
    zhs z ON z.uuid = c.uuid
INNER JOIN 
    cardIdentifiers ON cardIdentifiers.uuid = c.uuid
INNER JOIN
    sets ON sets.code = c.setCode
LEFT JOIN 
    sets parentSets ON parentSets.code = sets.parentCode"""

def escape_xml(s, without_quote=False):
    return s.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;").replace("\"", "&quot;") if not without_quote else s.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

def unescape_xml(s):
    return s.replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">").replace("&quot;", "\"").replace("\\n", "\n")

def read_table(sql_text):
    engine = create_engine(f'sqlite:////{DATABASEDIR}/zhs.sqlite')
    data = []
    with engine.connect() as connection:
        s = connection.execute(text(sql_text))
        for row in s:
            data.append({
                "scryfallId": row[0],
                "name": escape_xml(row[1]),
                "setCode": row[3],
                "number": row[4],
                "text": escape_xml(row[5].strip().replace(row[1], "CARDNAME") if row[5] else ""),
                "zhsName": escape_xml(row[7] if row[7] else ""),
                "zhsText": escape_xml(row[8].strip().replace(row[7], "CARDNAME") if row[8] else ""),
                "zhsUpdatedAt": row[9],
                "zhsUpdatedFrom": row[10],
                "translatedName": escape_xml(row[11] if row[11] else ""),
                "translatedText": escape_xml(row[12].strip().replace(row[7] or row[11] or "", "CARDNAME") if row[12] else ""),
                "translatedAt": row[13],
                "translatedFrom": row[14],
                "extra": row[15],
                "sortCode": f"{row[17]}-{row[16]}",
                "flavorText": escape_xml(row[18] if row[18] else ""),
                "zhsFlavorText": escape_xml(row[19] if row[19] else "")
            })
    return data

def sort_data_by_set(data):
    sorted_data = {}
    for row in data:
        if row['sortCode'] not in sorted_data:
            sorted_data[row['sortCode']] = []
        sorted_data[row['sortCode']].append(row)
    return sorted_data

def create_xliff_tree():
    '''
    创建一个xliff树，返回根节点
    '''
    root = etree.Element("xliff", xmlns="urn:oasis:names:tc:xliff:document:1.1")
    file_elem = etree.SubElement(root, "file")
    body = etree.SubElement(file_elem, "body")

    return root, body

def add_trans_unit(
    en_tree,
    zh_tree,
    data
):
    '''
    读取英文和中文的翻译，将其添加到对应的xliff树中。除了元数据外的内容都不是必须的。
    '''
    note = []
    if data['zhsUpdatedAt'] and data['zhsUpdatedFrom'] and data['setCode'] != data['zhsUpdatedFrom']:
        note.append(f"官方翻译于{data['zhsUpdatedFrom']}（{data['zhsUpdatedAt']}）")
    if data['translatedAt'] and data['translatedFrom']:
        note.append(f"非官方翻译于{data['translatedFrom']}（{data['translatedAt']}）")
    if data['extra']:
        note.append(data['extra'])
    note = "；".join(note)

    trans_unit = etree.SubElement(en_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Name", approved="yes")
    etree.SubElement(trans_unit, "source").text = data['name']
    etree.SubElement(trans_unit, "target", state="translated").text = data['name']

    if data['zhsName'] or data['translatedName']:
        if (data['translatedAt'] or "1900-01-01") > (data['zhsUpdatedAt'] or "1900-01-01"):
            zh_name = data['translatedName']
            zh_state = "needs-adaptation" if data['translatedFrom'] != "MTGZH" else "final"
            approved = "no" if data['translatedFrom'] != "MTGZH" else "yes"
        else:
            zh_name = data['zhsName']
            zh_state = "translated"
            approved = "no"
        trans_unit = etree.SubElement(zh_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Name", approved=approved)
        etree.SubElement(trans_unit, "source").text = data['name']
        etree.SubElement(trans_unit, "target", state=zh_state).text = zh_name
        if note: etree.SubElement(trans_unit, "note", nsmap={'from': "developer"}).text = note

    if data['text']:
        if (data['translatedAt'] or "1900-01-01") > (data['zhsUpdatedAt'] or "1900-01-01"):
            zh_text = data['translatedText']
            zh_state = "needs-adaptation" if data['translatedFrom'] != "MTGZH" else "final"
            approved = "no" if data['translatedFrom'] != "MTGZH" else "yes"
        elif data['zhsText']:
            zh_text = data['zhsText']
            zh_state = "translated"
            approved = "no"
        else:
            zh_text = ""
            
        if zh_text:
            # 如果存在中文，且中文和英文的行数无法对齐，则将所有行作为一个整体翻译（表现为Paragraphs）
            if zh_text.count("\\n") != data['text'].count("\\n"):
                trans_unit = etree.SubElement(en_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Text | Paragraphs", approved="yes")
                etree.SubElement(trans_unit, "source").text = data['text']
                etree.SubElement(trans_unit, "target", state="translated").text = data['text']

                trans_unit = etree.SubElement(zh_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Text | Paragraphs", approved=approved)
                etree.SubElement(trans_unit, "source").text = data['text']
                etree.SubElement(trans_unit, "target", state=zh_state).text = zh_text
                if note: etree.SubElement(trans_unit, "note", nsmap={'from': "developer"}).text = note
            # 如果中文和英文能够对齐，则将每行作为一个段落翻译
            else:
                for i, en_para, zh_para in zip(range(data['text'].count("\\n")+1), data['text'].split("\\n"), zh_text.split("\\n")):
                    trans_unit = etree.SubElement(en_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Text | Paragraph {i+1}", approved="yes")
                    etree.SubElement(trans_unit, "source").text = en_para
                    etree.SubElement(trans_unit, "target", state="translated").text = en_para

                    trans_unit = etree.SubElement(zh_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Text | Paragraph {i+1}", approved=approved)
                    etree.SubElement(trans_unit, "source").text = en_para
                    etree.SubElement(trans_unit, "target", state=zh_state).text = zh_para
                    if note: etree.SubElement(trans_unit, "note", nsmap={'from': "developer"}).text = note
        else:
            # 如果不存在中文，那就不需要考虑行数对齐的问题，直接分段落翻译
            for i, en_para in enumerate(data['text'].split("\\n")):
                trans_unit = etree.SubElement(en_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Card Text | Paragraph {i+1}", approved="yes")
                etree.SubElement(trans_unit, "source").text = en_para
                etree.SubElement(trans_unit, "target", state="translated").text = en_para

    if 'flavorText' in data and data['flavorText']:
        # 背景叙述永远作为一个整体翻译
        trans_unit = etree.SubElement(en_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Flavor Text", approved="yes")
        etree.SubElement(trans_unit, "source").text = data['flavorText']
        etree.SubElement(trans_unit, "target", state="translated").text = data['flavorText']

        if data['zhsFlavorText']:
            # 从数据库中取出的背景叙述均认为是被接受的（approved="yes"）
            trans_unit = etree.SubElement(zh_tree, "trans-unit", id=data['scryfallId'], resname=f"{data['setCode']}·{data['number']} | {data['name']} | Flavor Text", approved="yes")
            etree.SubElement(trans_unit, "source").text = data['flavorText']
            etree.SubElement(trans_unit, "target", state='translated').text = data['zhsFlavorText']
            if note: etree.SubElement(trans_unit, "note", nsmap={'from': "developer"}).text = note

def read_xliff_tree(file: str):
    '''
    将xliff树中的数据取出，返回原本的字典
    '''

    # 找到body节点
    tree = etree.parse(file)
    nsmap = {"ns": "urn:oasis:names:tc:xliff:document:1.1"}
    tree = tree.xpath("//ns:body", namespaces=nsmap)[0]
    tree = tree.xpath("/*/*/*/*")

    data = {}
    for node in tree:
        node_set_code_number, node_name, *node_types = node.attrib["resname"].split(" | ")
        if node_set_code_number not in data:
            data[node_set_code_number] = {
                "scryfallId": node.attrib["id"],
                "name": node_name,
                "setCode": node_set_code_number.split("·")[0],
                "number": node_set_code_number.split("·")[1],
                "extra": ""
            }
        card = data[node_set_code_number]
        target = node.xpath("ns:target", namespaces=nsmap)[0].text
        if node_types[0] == "Card Name":
            card['translatedName'] = unescape_xml(target)
        elif node_types[0] == "Card Text":
            if node_types[1] == "Paragraphs":
                card['translatedText'] = unescape_xml(target)
            else:
                if "translatedText" not in card:
                    card['translatedText'] = { node_types[1].split()[-1]: unescape_xml(target) }
                else:
                    card['translatedText'][node_types[1].split()[-1]] = unescape_xml(target)
        elif node_types[0] == "Flavor Text":
            card['zhsFlavorText'] = unescape_xml(target)

        # 预留备注的位置（暂时没有用上）
        '''
        is_note_from_developer = node.xpath("note")[0].attrib["xmlns:from"] == "developer"
        if is_note_from_developer:
            card['extra'] += unescape_xml(node.xpath("note")[0].text)
        '''

        card['translatedAt'] = datetime.fromtimestamp(os.path.getmtime(file)).strftime("%Y-%m-%d")
        card['translatedFrom'] = "MTGZH"

    # 字典形式的多行文本转化为字符串
    for card in data.values():
        if 'translatedText' in card and isinstance(card['translatedText'], dict):
            card['translatedText'] = "\\n".join([card['translatedText'][str(i+1)] for i in range(len(card['translatedText']))])

    return [card for card in data.values()]

REPLACED_SQL_TEXT = """SELECT
    *
FROM
    atomicCards
WHERE
    setCode = "{set_code}"
"""

UPDATE_NAME_TEXT = """UPDATE
    atomicCards
SET
    translatedName = :translatedName,
    translatedAt = :translatedAt,
    translatedFrom = :translatedFrom
WHERE
    name = :name
    AND setCode = :setCode
    AND number = :number
"""

UPDATE_TEXT_TEXT = """UPDATE
    atomicCards
SET
    translatedText = :translatedText,
    translatedAt = :translatedAt,
    translatedFrom = :translatedFrom
WHERE
    name = :name
    AND setCode = :setCode
    AND number = :number
"""

def replace_atomic_database(data, set_code):
    '''
    从xliff树中读取数据，将其写入atomicCards表
    目前需要传入set_code以确定要更新的数据
    '''
    engine = create_engine(f'sqlite:////{DATABASEDIR}/zhs.sqlite')
    with engine.connect() as connection:
        raw_data = connection.execute(text(REPLACED_SQL_TEXT.format(set_code=f'{set_code}'))).fetchall()
        raw_data = [dict(row) for row in raw_data]
    
    assert len(raw_data) == len(data), "数据长度不一致"
    
    data = sorted(data, key=lambda x: x['name'])
    raw_data = sorted(raw_data, key=lambda x: x['name'])

    assert [x['name'] for x in data] == [x['name'] for x in raw_data], "数据不一致"

    replacing_names = []
    replacing_texts = []
    for raw_card, updated_card in zip(raw_data, data):
        if updated_card['translatedAt'] < (raw_card['zhsUpdatedAt'] or ''):
            print(f"更新时间早于原数据更新时间，跳过：{updated_card['name']}")
            continue
        raw_name = raw_card['translatedName'] or raw_card['zhsName']
        raw_text = raw_card['translatedText'] or raw_card['zhsText']
        if updated_card.get('translatedName') and updated_card['translatedName'] != raw_name:
            print(f"将{updated_card['name']}的牌名加入更新队列")
            replacing_names.append(updated_card)
            card_name = updated_card['translatedName']
        else:
            card_name = raw_name
        if updated_card.get('translatedText') and updated_card['translatedText'] != raw_text:
            print(f"将{updated_card['name']}的文本加入更新队列")
            updated_card['translatedText'] = updated_card['translatedText'].replace("CARDNAME", card_name).replace("\n", "\\n")
            replacing_texts.append(updated_card)
            
    with engine.connect() as connection:
        for card in replacing_names:
            connection.execute(text(UPDATE_NAME_TEXT), card)
            print(f"已更新{card['name']}的牌名")
        for card in replacing_texts:
            connection.execute(text(UPDATE_TEXT_TEXT), card)
            print(f"已更新{card['name']}的文本")
        print(f"更新完成，更新了{len(replacing_names)}个牌名，{len(replacing_texts)}个文本")
        

## 以下代码将XLIFF文件导入到数据库中

In [7]:
data = read_xliff_tree("../project/lang/zh_Hans/1996-10-08-MIR-Oracle.xliff")
replace_atomic_database(data, "MIR")

/tmp/ipykernel_12155/3145834119.py:303: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  raw_data = [dict(row) for row in raw_data]


将Abyssal Hunter的文本加入更新队列
将Acidic Dagger的文本加入更新队列
将Afiya Grove的文本加入更新队列
将Afterlife的文本加入更新队列
将Agility的文本加入更新队列
将Alarum的文本加入更新队列
将Aleatory的文本加入更新队列
将Amber Prison的文本加入更新队列
将Amulet of Unmaking的文本加入更新队列
将Ancestral Memories的文本加入更新队列
将Armor of Thorns的文本加入更新队列
将Armorer Guildmage的文本加入更新队列
将Ashen Powder的文本加入更新队列
将Asmira, Holy Avenger的文本加入更新队列
将Auspicious Ancestor的文本加入更新队列
将Azimaet Drake的文本加入更新队列
将Bad River的文本加入更新队列
将Barbed Foliage的文本加入更新队列
将Barbed-Back Wurm的文本加入更新队列
将Barreling Attack的文本加入更新队列
将Basalt Golem的文本加入更新队列
将Bay Falcon的文本加入更新队列
将Bazaar of Wonders的文本加入更新队列
将Benevolent Unicorn的文本加入更新队列
将Benthic Djinn的文本加入更新队列
将Binding Agony的文本加入更新队列
将Blighted Shaman的文本加入更新队列
将Blind Fury的文本加入更新队列
将Blistering Barrier的文本加入更新队列
将Bone Harvest的文本加入更新队列
将Bone Mask的文本加入更新队列
将Breathstealer的文本加入更新队列
将Brushwagg的文本加入更新队列
将Builder's Bane的文本加入更新队列
将Burning Palm Efreet的文本加入更新队列
将Burning Shield Askari的文本加入更新队列
将Cadaverous Bloom的文本加入更新队列
将Cadaverous Knight的文本加入更新队列
将Canopy Dragon的文本加入更新队列
将Carrion的文本加入更新队列
将Catacomb Dragon的

## 以下代码从数据库中读取数据并生成xliff文件

In [ ]:
def export_xliff(sql_text, file_suffix):
    data = read_table(sql_text)

    EN_DIR = f"{WORKDIR}/project/lang/en"
    ZH_DIR = f"{WORKDIR}/project/lang/zh_Hans"

    sorted_data = sort_data_by_set(data)
    
    for set_code in sorted_data:
        en_tree, en_body = create_xliff_tree()
        zh_tree, zh_body = create_xliff_tree()

        for unit in sorted_data[set_code]:
            add_trans_unit(en_body, zh_body, unit)

        en_tree = etree.ElementTree(en_tree)
        zh_tree = etree.ElementTree(zh_tree)

        # if set_code == "CON": set_code = "CON_" # For some windows reasons
        
        en_tree.write(f"{EN_DIR}/{set_code}-{file_suffix}.xliff", pretty_print=True, xml_declaration=True, encoding="utf-8")
        zh_tree.write(f"{ZH_DIR}/{set_code}-{file_suffix}.xliff", pretty_print=True, xml_declaration=True, encoding="utf-8")

        print(f"Set {set_code} done.")

export_xliff(ATOMIC_SQL_TEXT, "Oracle")
export_xliff(ALL_SQL_TEXT, "Printed")
